# Matching

## Описание проекта

**Matching** - это задача поиска и сопоставления двух объектов из разных наборов данных. Такая потребность возникла у маркетплейса, который имеет большой перечен товаров для продажи. На склад стали поступать новые партии товаров, которые могут имеет небольшие отличия. Технически, это выглядит как совершенно новые товары. Фактически это те же товары, которые уже продаются на маркетплейсе. Задача сопоставить и связать новые товары со старыми, пользуясь совпадающими характеристиками.

**Что надо сделать?**
- предстоит реализовать финальную часть пайплайна матчинга. В ней необходимо принять решение для каждой пары (товар предлагаемый продавцом — товар на площадке), является ли она матчем или нет (бинарная классификация).
- Для этого у каждой пары есть набор признаков и наборы векторов (картиночные и текстовые), которые описывают товары из этой пары.
- В качестве метрики качества решения используется _F-score_.

**Данные:** [источник](https://www.kaggle.com/competitions/binary-classification-offers-on-the-marketplace/data)

- *train.csv -* обучающий датасет. Каждая строчка - один товар, для которого известен уникальный **id** (`0-query, 1-query, …`) , вектор признаков И id товара из *base.csv*, который максимально похож на него (по мнению экспертов).
- *test.csv* - датасет с товарами (уникальный **id** и вектор признаков), для которых надо найти наиболее близкие товары из *base.csv*
- *sample_submission.csv* - пример файла предсказаний.

Embeddings:

- `goods_image_vectors` и `offer_image_vectors` - содержат файлы с векторами изображений (embed_deperson.npy) и их идентификаторами (items_deperson.npy) для товаров ассортимента и предложений соответственно. Объекты в файлах соотносятся 1 к 1
- `goods_title_vectors` и `offer_title_vectors` - содержат файлы с векторами названий+атрибутов (embed_deperson.npy) и их идентификаторами (items_deperson.npy) для товаров ассортимента и предложений соответсвенно. Объекты в файлах соотносятся 1 к 1

**План работы:**
- Загрузить данные
- Понять задачу
- Подготовить данные
- Обучить модель (поиск ближайших соседей + ранжирование)
- Измерить качество
- git commit
- Сделать лучше!

Примечание: ячейки кода, исполнение которы занимает продолжительное время, имеют функцию `%%time` и время исполнения, указанное в конце выходных данных. 

___

## Подготовка

In [1]:
# импорт библиотек
#import phik
#import faiss
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# импорт спец. модулей
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, roc_curve, ConfusionMatrixDisplay

# константы
RANDOM_STATE = 42  # фиксация случайности
DATA_DIR = 'data/'

In [2]:
# чтение датасетов
train_df          = pd.read_csv(DATA_DIR + 'train.csv')
test_df           = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

loaded_array = np.load(DATA_DIR + 'sample_submission.csv')

CPU times: total: 2.39 s
Wall time: 2.39 s


___

## Анализ данных

Описание колонок:

- `offer_depersanalised` и `goods_depersanalised` - идентификаторы предложения и товара соответственно
- `sum_length` - суммарная длина пары названий и атрибутов в символах
- `dist` - расстояние между названиями предложения и товара *(заявлен, но отсутствует в исходных данных)*
- `attrs+title_score` - вероятность матча от рескоринговой модели
- `offer_price` и `item_price` - цена предложения и товара соответственно
- `goods_category_id` - категория товара
- `id` - идентификатор пары offer_depersanalised + $ + goods_depersanalised
- `target` (только в train.csv) - метка класса (0 - не матч, 1 - матч)

### Train

`Train` - набор из товаров новых поставок, для которых известны соответствующие товары из "старого" датасета base. ID этого товара содержится в отдельном датасете target.

In [3]:
# вывод общей информации
display(train_df.head())
train_df.info()
print('Кол-во товаров-дубликатов:', train_df.duplicated().sum())

# выделение цел. признака из обучающей выборки
train_targets = train_df['target']
train_df.drop('target', axis=1, inplace=True)

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,target,id
0,295140,1396793,37,0.027267,1070,NaN,14.0,0,295140$1396793
1,65291,1396586,38,0.050415,698,NaN,14.0,0,65291$1396586
2,39232,1396244,38,0.087280,837,NaN,14.0,0,39232$1396244
3,39232,1396513,38,0.087280,837,NaN,14.0,0,39232$1396513
4,65052,1396237,38,0.079773,1085,NaN,14.0,0,65052$1396237


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518441 entries, 0 to 2518440
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   offer_depersanalised  int64  
 1   goods_depersanalised  int64  
 2   sum_length            int64  
 3   attrs+title_score     float64
 4   offer_price           int64  
 5   goods_price           float64
 6   goods_category_id     float64
 7   target                int64  
 8   id                    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 172.9+ MB
Кол-во товаров-дубликатов: 0
CPU times: total: 2.58 s
Wall time: 2.63 s
